# Post-process vasca_pipe results
Make catalog file of selected sources only, associate and get store additional info for listed sources

In [1]:
region_name = "ALL_10-800_LOOSE" #"CAINGSGII_10-800"  #"TDS" #"CAINGSGII_10-800"  #"TDS" # "MDIS_10-800" # "TDS" # _ELAISN1
region_fname = "./vasca_pipeline/"+region_name+"/region_"+region_name+".fits"
region_cat_fname = region_fname.replace(".fits","_cat.fits")
do_cat = False                        # Recreate catalog file from vasca_pipe output region
do_cross_match = False                  # Does source matching with CDS
do_src_prep = True                   # Gets additional info for sources listes in srcs_ids list below.
do_variable_addition = False            # Adds variables of matched sources to tt_sources
do_LombScargle = False

srcs_ids = []
srcs_ids = [4993 ,13580 ,14504 ,30475 ,50388 ,52994 ,79477 ,120265 ,138783 ,139480 ,162697 ,204419 ,221349 ,228958 ,236839 ,253214 ,
            253670 ,256433 ,261213 ,331466 ,340464 ,381336 ,390864 ,489430 ,498065 ,525310 ,532554 ,533185 ,540235 ,583838 ,639720 ,
            645215 ,645730 ,659787 ,666224 ,673175 ,680119 ,691408 ,714140 ,764827 ,1037284 ,1069077 ,1070938 ,1805505 ,1839206
            ,2527679 ,3074917 ,] # WD ALL_10-800-LOOSE

In [2]:
%matplotlib widget
import ipywidgets as widgets
from IPython.display import display
import matplotlib.pyplot as plt
import numpy as np

from astropy.coordinates import SkyCoord
from astropy.table import unique, Table, hstack, join, setdiff
from astropy import units as uu
from astroquery.simbad import Simbad
from astropy.visualization import quantity_support
from astropy.modeling.models import BlackBody
from astropy import constants as cc
from matplotlib.ticker import ScalarFormatter
from astroquery.sdss import SDSS

from vasca.region import Region
from vasca.field import BaseField
import vasca.visualization as vvis
from vasca.tables import TableCollection, dd_vasca_columns
from vasca.utils import otype2ogroup,dd_ogrp2otypes, dd_ogrp2col, dd_filter2wavelength, mag2flux, flux2mag, query_vizier_sed, add_ogrp

from loguru import logger
logger.enable("vasca")

# Do catalog file

if do_cat:
    rg = Region()
    rg.load_from_fits(region_fname)
    rc = rg.get_region_catalog()
else:
    rc = Region()
    rc.load_from_fits(region_cat_fname)

#Cross match to SIMBAD and GAIA
if do_cross_match:
    rc.cross_match_cds(query_table="simbad")
    rc.cross_match_cds()

#Add variables
if do_variable_addition:
    rc.add_column("tt_gaiadr3", "Gmag_abs")
    sel_plx = rc.tt_gaiadr3["RPlx"]>3
    rc.tt_gaiadr3["Gmag_abs"][sel_plx]=rc.tt_gaiadr3["Gmag"][sel_plx] + 5.0 * np.log10(rc.tt_gaiadr3["Plx"][sel_plx]) - 10.0

    # Add object groups
    add_ogrp(rc.tt_simbad)
    add_ogrp(rc.tt_gaiadr3,provenance="GAIA")

if do_LombScargle:
    rc.set_LombScargle()

#Write out all changes to region
rc.write_to_fits(region_cat_fname, overwrite=True)

#Prepare sources, including Vizier SED.
if do_src_prep:
    for src_id in srcs_ids:
        tc_src = rc.get_src_from_id(src_id, load_from_file = False)
print("Done")

2023-10-18 18:27:11.220 | DEBUG    | vasca.tables:load_from_fits:330 - Loading file with name './vasca_pipeline/ALL_10-800_LOOSE/region_ALL_10-800_LOOSE_cat.fits'
2023-10-18 18:27:11.228 | DEBUG    | vasca.tables:load_from_fits:341 - Loading table 'tt_fields'
2023-10-18 18:27:11.240 | DEBUG    | vasca.tables:load_from_fits:341 - Loading table 'tt_filters'
2023-10-18 18:27:11.247 | DEBUG    | vasca.tables:load_from_fits:341 - Loading table 'tt_visits'
2023-10-18 18:27:11.254 | DEBUG    | vasca.tables:load_from_fits:341 - Loading table 'tt_sources'
2023-10-18 18:27:11.275 | DEBUG    | vasca.tables:load_from_fits:341 - Loading table 'tt_detections'
2023-10-18 18:27:11.313 | DEBUG    | vasca.tables:load_from_fits:341 - Loading table 'tt_coadd_detections'
2023-10-18 18:27:11.401 | DEBUG    | vasca.tables:load_from_fits:341 - Loading table 'tt_coadd_sources'
2023-10-18 18:27:11.419 | DEBUG    | vasca.tables:load_from_fits:341 - Loading table 'tt_src_id_map'
2023-10-18 18:27:11.427 | DEBUG   

Nr of spectra found 1


2023-10-18 18:27:14.591 | DEBUG    | vasca.tables:write_to_fits:264 - Writing table 'tt_lombscargle'
2023-10-18 18:27:14.598 | DEBUG    | vasca.tables:write_to_fits:264 - Writing table 'tt_visits'
2023-10-18 18:27:14.604 | DEBUG    | vasca.tables:write_to_fits:264 - Writing table 'tt_fields'
2023-10-18 18:27:14.616 | DEBUG    | vasca.tables:write_to_fits:264 - Writing table 'tt_filters'
2023-10-18 18:27:14.621 | DEBUG    | vasca.tables:write_to_fits:264 - Writing table 'tt_source_lc'
2023-10-18 18:27:14.634 | DEBUG    | vasca.tables:write_to_fits:264 - Writing table 'tt_sed'
2023-10-18 18:27:14.641 | DEBUG    | vasca.tables:write_to_fits:264 - Writing table 'tt_gphoton_lc'
2023-10-18 18:27:14.651 | DEBUG    | vasca.tables:write_to_fits:264 - Writing table 'tt_spectrum'
2023-10-18 18:27:14.799 | DEBUG    | vasca.region:get_src_from_id:338 - Getting Source object for rg_src_id: 14504
2023-10-18 18:27:14.802 | DEBUG    | vasca.tables:add_table:159 - Adding table 'tt_sources'
2023-10-18 18

Nr of spectra found 1


2023-10-18 18:27:15.755 | DEBUG    | vasca.tables:write_to_fits:264 - Writing table 'tt_lombscargle'
2023-10-18 18:27:15.761 | DEBUG    | vasca.tables:write_to_fits:264 - Writing table 'tt_visits'
2023-10-18 18:27:15.768 | DEBUG    | vasca.tables:write_to_fits:264 - Writing table 'tt_fields'
2023-10-18 18:27:15.779 | DEBUG    | vasca.tables:write_to_fits:264 - Writing table 'tt_filters'
2023-10-18 18:27:15.784 | DEBUG    | vasca.tables:write_to_fits:264 - Writing table 'tt_source_lc'
2023-10-18 18:27:15.795 | DEBUG    | vasca.tables:write_to_fits:264 - Writing table 'tt_sed'
2023-10-18 18:27:15.801 | DEBUG    | vasca.tables:write_to_fits:264 - Writing table 'tt_spectrum'
2023-10-18 18:27:15.926 | DEBUG    | vasca.region:get_src_from_id:338 - Getting Source object for rg_src_id: 30475
2023-10-18 18:27:15.928 | DEBUG    | vasca.tables:add_table:159 - Adding table 'tt_sources'
2023-10-18 18:27:15.958 | DEBUG    | vasca.tables:add_table:159 - Adding table 'tt_detections'
2023-10-18 18:27:1

...

Nr of spectra found 1


2023-10-18 18:27:27.080 | DEBUG    | vasca.tables:write_to_fits:264 - Writing table 'tt_lombscargle'
2023-10-18 18:27:27.088 | DEBUG    | vasca.tables:write_to_fits:264 - Writing table 'tt_visits'
2023-10-18 18:27:27.096 | DEBUG    | vasca.tables:write_to_fits:264 - Writing table 'tt_fields'
2023-10-18 18:27:27.115 | DEBUG    | vasca.tables:write_to_fits:264 - Writing table 'tt_filters'
2023-10-18 18:27:27.120 | DEBUG    | vasca.tables:write_to_fits:264 - Writing table 'tt_source_lc'
2023-10-18 18:27:27.132 | DEBUG    | vasca.tables:write_to_fits:264 - Writing table 'tt_sed'
2023-10-18 18:27:27.140 | DEBUG    | vasca.tables:write_to_fits:264 - Writing table 'tt_gphoton_lc'
2023-10-18 18:27:27.151 | DEBUG    | vasca.tables:write_to_fits:264 - Writing table 'tt_spectrum'
2023-10-18 18:27:27.306 | DEBUG    | vasca.region:get_src_from_id:338 - Getting Source object for rg_src_id: 236839
2023-10-18 18:27:27.309 | DEBUG    | vasca.tables:add_table:159 - Adding table 'tt_sources'
2023-10-18 1

Nr of spectra found 2


2023-10-18 18:27:54.754 | DEBUG    | vasca.region:get_src_from_id:338 - Getting Source object for rg_src_id: 1839206
2023-10-18 18:27:54.761 | DEBUG    | vasca.tables:add_table:159 - Adding table 'tt_sources'
2023-10-18 18:27:54.797 | DEBUG    | vasca.tables:add_table:159 - Adding table 'tt_detections'
2023-10-18 18:27:54.810 | DEBUG    | vasca.tables:add_table:159 - Adding table 'tt_simbad'
2023-10-18 18:27:54.832 | DEBUG    | vasca.tables:add_table:159 - Adding table 'tt_gaiadr3'
2023-10-18 18:27:54.844 | DEBUG    | vasca.tables:add_table:159 - Adding table 'tt_lombscargle'
2023-10-18 18:27:54.854 | DEBUG    | vasca.tables:add_table:159 - Adding table 'tt_visits'
2023-10-18 18:27:54.861 | DEBUG    | vasca.tables:add_table:159 - Adding table 'tt_fields'
2023-10-18 18:27:54.863 | DEBUG    | vasca.tables:add_table:159 - Adding table 'tt_filters'
2023-10-18 18:27:54.888 | DEBUG    | vasca.tables:add_table:159 - Adding table 'tt_source_lc'
2023-10-18 18:27:55.328 | DEBUG    | vasca.tables

Done


In [3]:
1*uu.Unit("1/d")


<Quantity 1. 1 / d>